<h1>Bayesian Personalization</h1>

Mathematically updates probability estimates of users interests based on their site activities using Bayesian inference. As more data comes in, the posterior purchase probabilities get revised dynamically.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix
from implicit.bpr import BayesianPersonalizedRanking

# Create a synthetic dataset of songs with genres
np.random.seed(42)
num_songs = 100
num_users = 50
interaction_density = 0.2# Density of interactions

song_ids = np.arange(num_songs)
user_ids = np.arange(num_users)



In [2]:
# Generate random song names and genres
song_names = [f"Song {i}" for i in range(num_songs)]
genres = ["Pop", "Rock", "Hip-Hop", "Electronic", "Country"] * (num_songs // 5)

# Create a DataFrame with random interactions (ratings)
data = []
for user_id in user_ids:
    listened_songs = np.random.choice(song_ids, size=int(interaction_density * num_songs), replace=False)
    ratings = np.random.randint(1, 6, size=len(listened_songs))
    data.extend(zip([user_id] * len(listened_songs), listened_songs, ratings))

song_df = pd.DataFrame(data, columns=["userId", "songId", "rating"])

# Add genres to the DataFrame
song_df["genre"] = [genres[song_id % len(genres)] for song_id in song_df["songId"]]
song_df.drop_duplicates(subset=['songId'], keep='first', inplace=True)
# Display the synthetic data
print("Synthetic Song Data:")
print(song_df.head(20))



Synthetic Song Data:
    userId  songId  rating       genre
0        0      83       3  Electronic
1        0      53       1  Electronic
2        0      70       3         Pop
3        0      45       3         Pop
4        0      44       1     Country
5        0      39       3     Country
6        0      22       5     Hip-Hop
7        0      80       2         Pop
8        0      10       2         Pop
9        0       0       1         Pop
10       0      18       4  Electronic
11       0      30       1         Pop
12       0      73       4  Electronic
13       0      33       2  Electronic
14       0      90       1         Pop
15       0       4       5     Country
16       0      76       3        Rock
17       0      77       4     Hip-Hop
18       0      12       3     Hip-Hop
19       0      31       3        Rock


In [3]:
# Split the data into training and testing sets
train, test = train_test_split(song_df, test_size=0.2, random_state=42)

# Create user-song matrix using 'rating' column values
user_song_matrix_train = pd.pivot_table(train, values='rating', index='userId', columns='songId', fill_value=0)


train_csr = csr_matrix(train.drop(columns=['genre']).values)
train_csr[:, 2] = 1  # Implicit feedback (interaction) represented by 1
train_csr = csr_matrix(train_csr)



In [4]:
# Create and train the BPR model
model = BayesianPersonalizedRanking(factors=64, iterations=50)
model.fit(train_csr)





  0%|          | 0/50 [00:00<?, ?it/s]

In [5]:
# Function to predict the next song based on Bayesian Personalization
def predict_next_song(user_history, model, song_df):
    user_id = user_history[0]  # Assuming a single user for simplicity

    # Get the user's posterior probabilities for songs
    user_posterior_probs = model.user_factors[user_id]

    # Exclude songs already listened to
    listened_songs = user_history
    user_posterior_probs[listened_songs] = 0

    # Find the index with the highest posterior probability as the next song
    next_song_index = np.argmax(user_posterior_probs)

    # Map the index back to the original song ID
    next_song_id = song_df.loc[next_song_index, 'songId']

    # Retrieve data related to the predicted next song from the DataFrame
    predicted_song_data = song_df[song_df['songId'] == next_song_id]

    return next_song_id, predicted_song_data



In [6]:
# Example: Predict the next song for a user with some listening history
user_history = [0, 5, 10]  # Assuming user has listened to songs with ids 0, 5, and 10
next_song_prediction, predicted_song_data = predict_next_song(user_history, model, song_df)

print(f"Predicted Next Song ID: {next_song_prediction}")
print("Predicted Song Data:")
print(predicted_song_data)

Predicted Next Song ID: 55
Predicted Song Data:
    userId  songId  rating genre
64       3      55       1   Pop
